<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/Poczenie-baz-danych-nagrd-z-main-data/colabs/Wczytanie_danych.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup sparka

# Załadowanie nowych baz

##Setup sparka

In [1]:
!pip install pyspark py4j
!pip install -q findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget ftp://ftp.ps.pl/pub/apache/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
!tar xf spark-3.3.1-bin-hadoop2.tgz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 43 kB/s 
     |████████████████████████████████| 200 kB 50.0 MB/s 
     |████████████████████████████████| 199 kB 52.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=cdc022104acdaee8e1c8ac1a6aa7ef678d30c3753b01b04b1923ccc23daf0306
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
--2022-11-23 15:02:11--  ftp://ftp.ps.pl/pub/apache/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
           => ‘spark-3.3.1-bin-hadoop2.tgz’
Resolving ftp.ps.pl (ftp.ps.pl)... 212.14.28.36, 2001:4c58:62:2::36
Connecting to ftp.ps.pl (ftp.ps.pl)|212.14.28.36|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/apache/spark/spark-3.3.1 ... done.
==>

In [2]:
import pyspark
import findspark
from pyspark.sql import SparkSession
import os

In [3]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop2"
spark=SparkSession.builder.appName('Colab').getOrCreate()
spark

#Pobranie danych

##Import danych

In [4]:
!wget https://datasets.imdbws.com/name.basics.tsv.gz
!wget https://datasets.imdbws.com/title.akas.tsv.gz
!wget https://datasets.imdbws.com/title.basics.tsv.gz
!wget https://datasets.imdbws.com/title.crew.tsv.gz
!wget https://datasets.imdbws.com/title.episode.tsv.gz
!wget https://datasets.imdbws.com/title.principals.tsv.gz
!wget https://datasets.imdbws.com/title.ratings.tsv.gz

--2022-11-23 15:02:41--  https://datasets.imdbws.com/name.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 99.84.160.41, 99.84.160.94, 99.84.160.101, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|99.84.160.41|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 236787831 (226M) [binary/octet-stream]
Saving to: ‘name.basics.tsv.gz’

name.basics.tsv.gz  100%[===================>] 225.82M  91.1MB/s    in 2.5s    

2022-11-23 15:02:43 (91.1 MB/s) - ‘name.basics.tsv.gz’ saved [236787831/236787831]

--2022-11-23 15:02:44--  https://datasets.imdbws.com/title.akas.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 99.84.160.41, 99.84.160.94, 99.84.160.101, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|99.84.160.41|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 288369065 (275M) [binary/octet-stream]
Saving to: ‘title.akas.tsv.gz’

title.akas.tsv.gz   100%[===================>] 275.01M  33.

##Rozpakowanie danych

In [5]:
!gzip -dc /content/name.basics.tsv.gz > name.basics.csv
!gzip -dc /content/title.akas.tsv.gz > title.akas.csv
!gzip -dc /content/title.basics.tsv.gz > title.basic.csv
!gzip -dc /content/title.crew.tsv.gz > title.crew.csv
!gzip -dc /content/title.episode.tsv.gz > title.episode.csv
!gzip -dc /content/title.principals.tsv.gz > title.principals.csv
!gzip -dc /content/title.ratings.tsv.gz > title.ratings.csv

#Wczytanie danych

##Wczytajmy dane z rozpakowanych plików

In [6]:
df_name_basics=spark.read.option("header", "true").option("delimiter", "\t").csv('name.basics.csv' ) 
df_title_akas=spark.read.option("header","true").option("delimiter", "\t").csv('title.akas.csv')
df_title_basic=spark.read.option("header","true").option("delimiter", "\t").csv('title.basic.csv')
df_title_crew=spark.read.option("header","true").option("delimiter", "\t").csv('title.crew.csv')
df_title_episode=spark.read.option("header","true").option("delimiter","\t").csv('title.episode.csv')
df_title_principals=spark.read.option("header","true").option("delimiter","\t").csv('title.principals.csv')
df_title_ratings=spark.read.option("header","true").option("delimiter","\t").csv('title.ratings.csv')

##Wyświetlmy dane

In [7]:
df_name_basics.show(3)
df_title_akas.show(3)
df_title_basic.show(3)
df_title_crew.show(3)
df_title_episode.show(3)
df_title_principals.show(3)
df_title_ratings.show(3)

+---------+---------------+---------+---------+--------------------+--------------------+
|   nconst|    primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+---------------+---------+---------+--------------------+--------------------+
|nm0000001|   Fred Astaire|     1899|     1987|soundtrack,actor,...|tt0031983,tt00723...|
|nm0000002|  Lauren Bacall|     1924|     2014|  actress,soundtrack|tt0117057,tt00373...|
|nm0000003|Brigitte Bardot|     1934|       \N|actress,soundtrac...|tt0054452,tt00491...|
+---------+---------------+---------+---------+--------------------+--------------------+
only showing top 3 rows

+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|  titleId|ordering|               title|region|language|      types|   attributes|isOriginalTitle|
+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|tt0000001|       1|          Карменсіта|    

# Złączenie istotnych kolumn

## Wybierzmy następujące kolumny z tabel:
- df_title_basic:
  - tconst (unikalny numer tytułu - potrzebne do złączenia tabel)
  - titleType (rodzaj tytułu, np. film, serial - można porównywać podobieństwo aktorów na podstawie tego w jakich typach produkcji grali)
  - originalTitle (tytuł produkcji - być może przyda się do analizy podobnych filmów)
  - isAdult (czy produkcja jest dla dorosłych - można wykorzystać do porównywania aktorów na podstawie tego czy grają głównie w produkcjach dla dorosłych)
  - genres (gatunki - może posłużyć do porównania aktorów na podstawie tego, że grali w produkcjach o podobnych gatunkach)

- df_title_principals:
  - tconst (unikalny numer tytułu - potrzebne do złączenia tabel)
  - nconst (unikalny numer aktora - potrzebne do złączenia tabel)
  - category (kategoria pracy (actor/actress) - może posłużyć do porównania aktorów na podstawie takiej samej płci)
  - characters (postacie zagrane w produkcji - może posłużyć do porównania aktorów na podstawie podobnych granych postaci)

- df_name_basics:
  - nconst(unikalny numer aktora - potrzebne do złączenia tabel)
  - primaryName (imię i nazwisko aktora - potrzebne do wyświetlania aktora lub do odnajdywania go w bazie po imieniu i nazwisku)
  - birthYear (rok urodzenia aktora - może posłużyć do porównania aktorów na podstawie podobnych okresów, w których żyli i grali)
  - deathYear ( rok śmierci aktora - może posłużyć do porównania aktorów na podstawie podobnych okresów, w których żyli i grali)
  - knownForTitles (tytuły produkcji, z których znany jets aktor - może posłużyć do porównania aktorów na podstawie tych samych lub podobnych produkcji, z których są znani)

In [8]:
df_title_basic_selected = df_title_basic.select(["tconst", "titleType", "originalTitle", "isAdult", "genres"])
df_title_principals_selected = df_title_principals.select(["tconst", "nconst", "category", "characters"])
df_name_basics_selected = df_name_basics.select(["nconst", "primaryName", "birthYear", "deathYear", "knownForTitles"])

## Wybierzmy z tabeli df_title_principals_selected tylko aktorów i aktorki

In [9]:
df_title_principals_selected = df_title_principals_selected.filter((df_title_principals_selected.category == "actor") | (df_title_principals_selected.category == "actress"))

# Złączmy tabele df_name_basics, df_title_principals i df_title_basic

In [10]:
print("df_name_basics_selected dataframe size: ", (df_name_basics_selected.count(), len(df_name_basics_selected.columns)))
print("df_title_principals_selected dataframe size: ", (df_title_principals_selected.count(), len(df_title_principals_selected.columns)))
print("df_title_basic_selected dataframe size: ", (df_title_basic_selected.count(), len(df_title_basic_selected.columns)))
data = df_name_basics_selected.join(df_title_principals_selected, "nconst", "inner")
print("joined dataframe size: ", (data.count(), len(data.columns)))
data = data.join(df_title_basic_selected, "tconst", "left")
print("joined dataframe size: ", (data.count(), len(data.columns)))

df_name_basics_selected dataframe size:  (12096703, 5)
df_title_principals_selected dataframe size:  (20815083, 4)
df_title_basic_selected dataframe size:  (9389356, 5)
joined dataframe size:  (20809855, 8)
joined dataframe size:  (20809855, 12)


# Wybierzmy przykładowe wiersze z końcowego dataframe'u

In [11]:
#sample = data.rdd.takeSample(False, 5)

In [12]:
import pandas as pd
#sample = pd.DataFrame(sample, columns = data.columns)
#sample

# Załadowanie nowych baz

   **W celu załadowania nowych danych należy:**


1. Pobrać token API:
  * Wejść w *zespół Aktorzy* na teams
   * Wybrać zakładkę *pliki*
    * Pobrać plik *kaggle.json* - API
2. Utworzyć plik na swoim dysku pod nazwą "Big Data"
3. Wrzucić pobrane API na swój dysk
4. Uruchomić kod :)




## Wczytajmy bazę oskarów: "Best Movies Watchlist"


In [42]:
from google.colab import drive
drive.mount('/content/gdrive')

#%cd /content/gdrive/MyDrive/Big Data/kaggle.json
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Big Data"

#from google.colab import drive
#drive.mount('/content/drive')
#random.seed(123456)
#np.set_printoptions(precision=6, suppress=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
! pip install kaggle
! mkdir ~/.kaggle #Utwórz katalog o nazwie „.kaggle”
! cp kaggle.json ~/.kaggle/ # Skopiuj plik „kaggle.json” do tego nowego katalogu
! chmod 600 ~/.kaggle/kaggle.json #Przydziel wymagane uprawnienia do tego pliku.

In [44]:
#import pandas as pd

! kaggle datasets download unanimad/the-oscar-award #załadujmy dane
! unzip the-oscar-award.zip #rozpakujmy z zip'a
oscars=spark.read.option("header","true").csv('the_oscar_award.csv')
#oscars=pd.read_csv('the_oscar_award.csv')
oscars.show(5)


the-oscar-award.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  the-oscar-award.zip
replace the_oscar_award.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: the_oscar_award.csv     


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.7/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

Main Data:
* primaryName	- aktor
* originalTitle - nazwa filmu

W nowej bazie oskarów:
* name - aktor
* film - film

## Wczytajmy bazę złotych globów: "Golden Globe Awards, 1944 - 2020"

In [16]:
! kaggle datasets download unanimad/golden-globe-awards #załadujmy dane
! unzip golden-globe-awards.zip #rozpakujmy z zip'a
globe=spark.read.option("header","true").csv('golden_globe_awards.csv')
globe.show(5)

  0% 0.00/117k [00:00<?, ?B/s]
100% 117k/117k [00:00<00:00, 27.9MB/s]
Archive:  golden-globe-awards.zip
  inflating: golden_globe_awards.csv  
+---------+----------+--------+--------------------+--------------------+--------------------+----+
|year_film|year_award|ceremony|            category|             nominee|                film| win|
+---------+----------+--------+--------------------+--------------------+--------------------+----+
|     1943|      1944|       1|Best Performance ...|      Katina Paxinou|For Whom The Bell...|True|
|     1943|      1944|       1|Best Performance ...|       Akim Tamiroff|For Whom The Bell...|True|
|     1943|      1944|       1|Best Director - M...|          Henry King|The Song Of Berna...|True|
|     1943|      1944|       1|             Picture|The Song Of Berna...|                null|True|
|     1943|      1944|       1|Actress In A Lead...|      Jennifer Jones|The Song Of Berna...|True|
+---------+----------+--------+--------------------+-----

## Wczytajmy bazę złotych globów: "Primetime Emmy Awards, 1949-2017"

In [41]:
! kaggle datasets download unanimad/emmy-awards #załadujmy dane
! unzip emmy-awards.zip #rozpakujmy z zip'a
emmy_awards_category=spark.read.option("header","true").csv('emmy_awards_categories.csv')
emmy_awards_category.show(5)


emmy-awards.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  emmy-awards.zip
replace emmy_awards_categories.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: emmy_awards_categories.csv  
replace the_emmy_awards.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: the_emmy_awards.csv     


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.7/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [ ]:
emmy_awards=spark.read.option("header","true").csv('the_emmy_awards.csv')
emmy_awards.show(5)

## Zaczytanie danych TMDB 

   informacje o aktorach i osobach pracujących przy produkcji filmów oraz budżetach, gatunkach, popularności, wytwórniach filmowych, oryginalnyh językach i tytułach


In [18]:
#!kaggle kernels init -p tmdb/tmdb-movie-metadata #wygenerować metadane
! kaggle datasets download tmdb/tmdb-movie-metadata #załadujmy dane
! unzip tmdb-movie-metadata.zip
tmdb_credits=spark.read.option("header","true").csv('tmdb_5000_credits.csv')
tmdb_movies=spark.read.option("header","true").csv('tmdb_5000_movies.csv')

  0% 0.00/8.89M [00:00<?, ?B/s]
100% 8.89M/8.89M [00:00<00:00, 143MB/s]
Archive:  tmdb-movie-metadata.zip
  inflating: tmdb_5000_credits.csv   
  inflating: tmdb_5000_movies.csv    


In [19]:
tmdb_credits.show(5)

+--------+--------------------+-------------------+--------------------+
|movie_id|               title|               cast|                crew|
+--------+--------------------+-------------------+--------------------+
|   19995|              Avatar|"[{""cast_id"": 242| ""character"": "...|
|     285|Pirates of the Ca...|  "[{""cast_id"": 4| ""character"": "...|
|  206647|             Spectre|  "[{""cast_id"": 1| ""character"": "...|
|   49026|The Dark Knight R...|  "[{""cast_id"": 2| ""character"": "...|
|   49529|         John Carter|  "[{""cast_id"": 5| ""character"": "...|
+--------+--------------------+-------------------+--------------------+
only showing top 5 rows



In [20]:
tmdb_movies.show(5)

+---------+-------------+--------------------+------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+--------------------+--------------+--------------------+--------------+--------------------+---------------+--------------------+----------------+
|   budget|       genres|            homepage|          id|            keywords|original_language|      original_title|            overview|          popularity|production_companies|production_countries|   release_date|             revenue|       runtime|    spoken_languages|        status|             tagline|          title|        vote_average|      vote_count|
+---------+-------------+--------------------+------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+--------------------+--------------+-----------

# Połączmy dane na temat nagród z głównymi danymi

In [34]:
oscars_selected = oscars.filter((oscars.category.like('%ACTOR%')) | (oscars.category.like('%ACTRESS%')))
globe_selected = globe.filter((globe.category.like('%Actor%')) | (globe.category.like('%Actress%')))

In [37]:
oscars_selected = oscars.select(["year_ceremony", "category", "name", "film", "winner"])
globe_selected = globe.select(["year_award", "category", "nominee", "film", "win"])
emmy_awards_selected = emmy_awards.select(["name", "category", "role"])

In [38]:
oscars_selected = oscars_selected.withColumnRenamed("year_ceremony","year_oscars").withColumnRenamed("category","category_oscars").withColumnRenamed("winner","winner_oscars")

globe_selected = globe_selected.withColumnRenamed("year_award","year_globes").withColumnRenamed("category","category_globes").withColumnRenamed("win","win_globes")

emmy_awards_selected = emmy_awards_selected.withColumnRenamed("category","category_emmy").withColumnRenamed("role","role_emmy")

In [39]:
print("data dataframe size: ", (data.count(), len(data.columns)))
print("oscars_selected dataframe size: ", (oscars_selected.count(), len(oscars_selected.columns)))
print("globe_selected dataframe size: ", (globe_selected.count(), len(globe_selected.columns)))
print("emmy_awards_selected dataframe size: ", (emmy_awards_selected.count(), len(emmy_awards_selected.columns)))
data = data.join(oscars_selected, (data.primaryName == oscars_selected.name) & (data.primaryTitle == oscars_selected.film), "left")
print("joined dataframe size: ", (data.count(), len(data.columns)))
data = data.join(globe_selected, (data.primaryName == globe_selected.nominee) & (data.primaryTitle == globe_selected.film), "left")
print("joined dataframe size: ", (data.count(), len(data.columns)))
data = data.join(emmy_awards_selected, data.primaryName == emmy_awards_selected.name, "left")
print("joined dataframe size: ", (data.count(), len(data.columns)))

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.7/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [ ]:
print(oscars_selected.filter(~oscars_selected.winner_oscars.isNull()).count())
print(globe_selected.filter(~globe_selected.win_globes.isNull()).count())
print(emmy_awards_selected.filter(~emmy_awards_selected.category_emmy.isNull()).count())